In [1]:
from dask.distributed import Client, LocalCluster
import logging

cluster = LocalCluster(
    n_workers=28,
    threads_per_worker=8,
    silence_logs=logging.DEBUG
)

client = Client(cluster, heartbeat_interval=10000)

print(client.dashboard_link)

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:     tcp://127.0.0.1:40233
distributed.scheduler - INFO -   dashboard at:            127.0.0.1:8787
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:38911'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:42289'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:39027'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:40877'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:37323'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:39631'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:43873'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:45981'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:34505'
distributed.nanny - INFO -         Start Nanny at: 'tcp://127.0.0.1:39689'
distributed.nanny - INFO -         Start Nanny at: 'tcp:

http://127.0.0.1:8787/status


In [2]:
import afqinsight as afqi
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os.path as op
import pandas as pd
import pickle
import seaborn as sns

from datetime import datetime

from sklearn.base import clone
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import median_absolute_error, r2_score
from sklearn.metrics import explained_variance_score, mean_squared_error

from skopt import BayesSearchCV
from skopt.plots import plot_convergence, plot_objective, plot_evaluations

print(afqi.__version__)

0.2.9.dev460469908


In [3]:
X, y, groups, columns, subjects, classes = afqi.load_afq_data(
    "../data/raw/hbn_data",
    target_cols=["Age"],
    index_col="EID"
)

In [4]:
X.shape

(1597, 3600)

In [5]:
df_y = pd.read_csv("../data/raw/hbn_data/subjects.csv").drop(axis="rows", index=0)
df_y = df_y[["EID", "Age"]]
df_y = df_y.set_index("EID", drop=True)
df_subs = pd.DataFrame(index=subjects)
df_subs = df_subs.merge(df_y, how="left", left_index=True, right_index=True)
y = df_subs["Age"].astype(np.float64).values
nan_mask = np.logical_not(np.isnan(y))
y = y[nan_mask]
X = X[nan_mask, :]

In [6]:
print(X.shape)
print(y.shape)

(978, 3600)
(978,)


In [7]:
len(groups)

36

In [8]:
def get_cv_results(n_repeats=5, n_splits=10,
                   power_transformer=False, 
                   shuffle=False,
                   ensembler=None,
                   target_transform_func=None,
                   target_transform_inverse_func=None,
                   n_estimators=10,
                   trim_nodes=0,
                   square_features=False):
    if shuffle:
        rng = np.random.default_rng()
        y_fit = rng.permutation(y)
    else:
        y_fit = np.copy(y)
    
    if trim_nodes > 0:
        grp_mask = np.zeros_like(groups[0], dtype=bool)
        grp_mask[trim_nodes:-trim_nodes] = True
        X_mask = np.concatenate([grp_mask] * len(groups))

        groups_trim = []
        start_idx = 0
        
        for grp in groups:
            stop_idx = start_idx + len(grp) - 2 * trim_nodes
            groups_trim.append(np.arange(start_idx, stop_idx))
            start_idx += len(grp) - 2 * trim_nodes
            
        X_trim = X[:, X_mask]
    elif trim_nodes == 0:
        groups_trim = [grp for grp in groups]
        X_trim = np.copy(X)
    else:
        raise ValueError("trim_nodes must be non-negative.")
        
    if square_features:
        _n_samples, _n_features = X_trim.shape
        X_trim = np.hstack([X_trim, np.square(X_trim)])
        groups_trim = [np.concatenate([g, g + _n_features]) for g in groups_trim]

    cv = RepeatedKFold(
        n_splits=n_splits,
        n_repeats=n_repeats,
        random_state=1729
    )

    cv_results = {}
    pipe_skopt = afqi.make_afq_regressor_pipeline(
        imputer_kwargs={"strategy": "median"},
        use_cv_estimator=True,
        power_transformer=power_transformer,
        scaler="standard",
        groups=groups_trim,
        verbose=0,
        pipeline_verbosity=False,
        tuning_strategy="bayes",
        cv=3,
        n_bayes_points=9,
        n_jobs=28,
        l1_ratio=[0.0, 1.0],
        eps=5e-2,
        n_alphas=100,
        ensemble_meta_estimator=ensembler,
        ensemble_meta_estimator_kwargs={
            "n_estimators": n_estimators,
            "n_jobs": 1,
            "oob_score": True,
            "random_state": 1729,
        },
        target_transform_func=target_transform_func,
        target_transform_inverse_func=target_transform_inverse_func,
    )

    for cv_idx, (train_idx, test_idx) in enumerate(cv.split(X_trim, y_fit)):
        start = datetime.now()

        X_train, X_test = X_trim[train_idx], X_trim[test_idx]
        y_train, y_test = y_fit[train_idx], y_fit[test_idx]

        with joblib.parallel_backend("dask"):
            pipe_skopt.fit(X_train, y_train)

        cv_results[cv_idx] = {
            "pipeline": pipe_skopt,
            "train_idx": train_idx,
            "test_idx": test_idx,
            "y_pred": pipe_skopt.predict(X_test),
            "y_true": y_test,
            "test_mae": median_absolute_error(y_test, pipe_skopt.predict(X_test)),
            "train_mae": median_absolute_error(y_train, pipe_skopt.predict(X_train)),
            "test_r2": r2_score(y_test, pipe_skopt.predict(X_test)),
            "train_r2": r2_score(y_train, pipe_skopt.predict(X_train)),
        }
        
        if ((target_transform_func is not None)
            or (target_transform_inverse_func is not None)):
            cv_results[cv_idx]["coefs"] = [
                est.coef_ for est
                in pipe_skopt.named_steps["estimate"].regressor_.estimators_
            ]
            cv_results[cv_idx]["alpha"] = [
                est.alpha_ for est
                in pipe_skopt.named_steps["estimate"].regressor_.estimators_
            ]
            cv_results[cv_idx]["l1_ratio"] = [
                est.l1_ratio_ for est
                in pipe_skopt.named_steps["estimate"].regressor_.estimators_
            ]
        else:
            cv_results[cv_idx]["coefs"] = [
                est.coef_ for est
                in pipe_skopt.named_steps["estimate"].estimators_
            ]
            cv_results[cv_idx]["alpha"] = [
                est.alpha_ for est
                in pipe_skopt.named_steps["estimate"].estimators_
            ]
            cv_results[cv_idx]["l1_ratio"] = [
                est.l1_ratio_ for est
                in pipe_skopt.named_steps["estimate"].estimators_
            ]

        if ensembler is None:
            if ((target_transform_func is not None)
                or (target_transform_inverse_func is not None)):
                cv_results[cv_idx]["optimizer"] = pipe_skopt.named_steps["estimate"].regressor_.bayes_optimizer_                
            else:
                cv_results[cv_idx]["optimizer"] = pipe_skopt.named_steps["estimate"].bayes_optimizer_

        print(f"CV index [{cv_idx:3d}], Elapsed time: ", datetime.now() - start)
        
    return cv_results, y_fit

In [9]:
results = {}

for trim_nodes in range(5):
    results[f"bagging_target_transform_trim{trim_nodes}"] = get_cv_results(
        n_splits=5, n_repeats=1, power_transformer=False,
        ensembler="serial-bagging", shuffle=False, n_estimators=20,
        target_transform_func=np.log, target_transform_inverse_func=np.exp,
        trim_nodes=trim_nodes, square_features=False
    )
    
    results[f"bagging_target_transform_trim{trim_nodes}_square_features"] = get_cv_results(
        n_splits=5, n_repeats=1, power_transformer=False,
        ensembler="serial-bagging", shuffle=False, n_estimators=20,
        target_transform_func=np.log, target_transform_inverse_func=np.exp,
        trim_nodes=trim_nodes, square_features=True
    )


# with open("hbn_regression.pkl", "rb") as fp:
#     results = pickle.load(fp)

distributed.scheduler - INFO - Receive client connection: Client-worker-6412be1c-51b1-11eb-b865-ef2b9c0b7a78
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Receive client connection: Client-worker-64143ad2-51b1-11eb-b79e-c3725187cb0b
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Receive client connection: Client-worker-64145050-51b1-11eb-b708-19ece6f5f154
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Receive client connection: Client-worker-6415879c-51b1-11eb-baa2-b1162cc68753
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Receive client connection: Client-worker-64158f02-51b1-11eb-ba40-8b044ac6f5a1
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Receive client connection: Client-worker-6416eddc-51b1-11eb-b931-2dc0b6fc4e77
distributed.core - INFO - Starting established connectio

CV index [  0], Elapsed time:  0:54:12.136269


distributed.core - INFO - Event loop was unresponsive in Nanny for 5.86s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 5.87s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 5.87s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 5.87s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 5.87s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This c

CV index [  1], Elapsed time:  0:53:45.650362


/opt/tljh/user/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/opt/tljh/user/lib/python3.7/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
distributed.core - INFO - Event loop was unresponsive in Nanny for 5.04s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 5.05s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 5.04s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts an

...

CV index [  1], Elapsed time:  1:21:38.155861


distributed.core - INFO - Event loop was unresponsive in Nanny for 8.57s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.59s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.59s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.60s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.60s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This c

CV index [  3], Elapsed time:  1:23:26.027061


distributed.core - INFO - Event loop was unresponsive in Nanny for 8.42s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.42s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.42s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.43s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 8.43s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This c

CV index [  4], Elapsed time:  1:29:29.964131


In [16]:
with open("hbn_regression_paper.pkl", "wb") as fp:
    pickle.dump(results, fp)

In [17]:
results.keys()

dict_keys(['bagging_target_transform_trim0', 'bagging_target_transform_trim0_square_features', 'bagging_target_transform_trim1', 'bagging_target_transform_trim1_square_features', 'bagging_target_transform_trim2', 'bagging_target_transform_trim2_square_features', 'bagging_target_transform_trim3', 'bagging_target_transform_trim3_square_features', 'bagging_target_transform_trim4', 'bagging_target_transform_trim4_square_features'])

distributed.core - INFO - Event loop was unresponsive in Nanny for 9.61s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 9.61s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 9.61s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 9.62s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This can cause timeouts and instability.
distributed.core - INFO - Event loop was unresponsive in Nanny for 9.62s.  This is often caused by long-running GIL-holding functions or moving large chunks of data. This c

In [15]:
for metric in ["test_mae", "test_r2", "train_mae", "train_r2"]:
    for key, res in results.items():
        mean_metric = [cvr[metric] for cvr in res[0].values()]
        print(metric, key, np.mean(mean_metric))
    
    print()

test_mae bagging_target_transform_trim0 1.5171607595653864
test_mae bagging_target_transform_trim0_square_features 1.5562234930686443
test_mae bagging_target_transform_trim1 1.5263564341497706
test_mae bagging_target_transform_trim1_square_features 1.5361760308208654
test_mae bagging_target_transform_trim2 1.533763148314954
test_mae bagging_target_transform_trim2_square_features 1.5362001700564134
test_mae bagging_target_transform_trim3 1.5522562533499642
test_mae bagging_target_transform_trim3_square_features 1.5398333144787177
test_mae bagging_target_transform_trim4 1.5551844413360971
test_mae bagging_target_transform_trim4_square_features 1.5495836356424257

test_r2 bagging_target_transform_trim0 0.5594104657066131
test_r2 bagging_target_transform_trim0_square_features 0.5041686376532943
test_r2 bagging_target_transform_trim1 0.549253065703965
test_r2 bagging_target_transform_trim1_square_features 0.49328235846299007
test_r2 bagging_target_transform_trim2 0.541540846493141
test_r2 b